##### Find tweets based on keywowrds
- The code below, fetches tweets according to the given keywords...

In [10]:
import tweepy
import re
import pandas as pd
from deep_translator import GoogleTranslator 
from googletrans import Translator
translator = Translator()
keywords = ["education", "women", "woman",
            "schools", "school", "univesities",
            "university", "closed","close",
            "ban", "banned", "from school",
            "girls", "girl","work"
            "worked","edu","teachers", "teacher","learning",
            "آموزش", "زنان" "زن", "مدارس", "مدرسه", "دانشگاه ها",
            "دانشگاه", "بسته", "بسته",
            "ممنوعیت", "ممنوع",
            "از مدرسه", "دختران", "دختر", "کار",
            "کار کرد", "آموزش", "معلمان", "معلم", "یادگیری"]
# Authenticate your API key
consumer_key= "dqOPlo8TYO2Bo6P4Uo5sAkmes"
consumer_secret= "3TUe2RlqY5BT6LnvtUAA0Kf1mPO8k7MH3ZLxxELcsj7OxG8doh"
access_token= "2585433478-GoThRI4yGSKJX3tO8GX2onBsVdZ9EyPP1uL7dSi"
access_token_secret= "WBHrZjnz8lDrKAlLpF0uNKMydws4iodCuWUtHwdqmYoq1"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
# Construct a query to fetch tweets from "@user"
# Construct a query to fetch tweets related to education in English
tweets = []
for tweet in tweepy.Cursor(api.user_timeline, screen_name="@baharmhr", count=3000).items():
    # Filter out tweets that are not related to any of the keywords
    if any(keyword in tweet.text.lower() for keyword in keywords):
        tweets.append(tweet)

# Create a list of dictionaries containing the relevant tweet data
tweet_data = []
for tweet in tweets:
    tweet_dict = {
        "text": tweet.text,
        "name": tweet.user.screen_name,
        "created_at": tweet.created_at.strftime("%Y-%m-%d %H:%M:%S")
    }
    tweet_data.append(tweet_dict)
# Create a Pandas DataFrame from the tweet data
df = pd.DataFrame(tweet_data)

#remove and mentions
def remove_usernames_links(tweet):
    tweet = re.sub("@[^\s]+","",str(tweet))
    tweet = re.sub("http[^\s]+","",str(tweet))
    # tweet = re.sub(r"[RT]+","",str(tweet))
    return tweet
df = df.replace(r"\n"," ", regex=True) 
df["text"] = df["text"].apply(remove_usernames_links)
# Define a function to remove emojis from a string
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r"", text)
# Apply the function to a Pandas DataFrame column
df["text"] = df["text"].apply(remove_emoji)
#translate

df["text"] = df["text"].apply(lambda x: GoogleTranslator(source="auto", target="en").translate(x))

if "gender" not in df.columns:
    df["gender"] = ""
if "ethnic" not in df.columns:
    df["ethnic"] = ""
mask = (df["gender"] == "") & (df["ethnic"] == "")
df["gender"] = "male" 
df["ethnic"] = "uzbek"
# Print the DataFrame
df.to_excel("../data_processed/individual/@baharmhr_men_uzbek.xlsx", index=False)